# This notebook is used to generate images and masks to train the model

#### Note: Check the paths to save and read images before running

## Cropping - Displays image and user has to crop by using mouse

In [1]:
import os
import cv2
import numpy as np

img_dir = "ex/img/"
dest_dir = "ex/cropped/"
os.makedirs(dest_dir, exist_ok=True)

# Loop over images in directory
for filename in os.listdir(img_dir):
    cropping = False
    x_start, y_start, x_end, y_end = 0, 0, 0, 0
    
    # Load image
    img_path = os.path.join(img_dir, filename)
    image = cv2.imread(img_path)
    oriImage = image.copy()
    
    def mouse_crop(event, x, y, flags, param):
        # grab references to the global variables
        global x_start, y_start, x_end, y_end, cropping

        # if the left mouse button was DOWN, start RECORDING
        # (x, y) coordinates and indicate that cropping is being
        if event == cv2.EVENT_LBUTTONDOWN:
            x_start, y_start, x_end, y_end = x, y, x, y
            cropping = True

        # Mouse is Moving
        elif event == cv2.EVENT_MOUSEMOVE:
            if cropping == True:
                x_end, y_end = x, y

        # if the left mouse button was released
        elif event == cv2.EVENT_LBUTTONUP:
            # record the ending (x, y) coordinates
            x_end, y_end = x, y
            cropping = False # cropping is finished

            refPoint = [(x_start, y_start), (x_end, y_end)]

            if len(refPoint) == 2: #when two points were found
                
                roi = oriImage[refPoint[0][1]:refPoint[1][1], refPoint[0][0]:refPoint[1][0]]
                cv2.imshow("Cropped", roi)
            
            dest_path = os.path.join(dest_dir, filename[:-4] + ".tiff")
            cv2.imwrite(dest_path, roi)

    cv2.namedWindow("image")
    cv2.setMouseCallback("image", mouse_crop)

    while True:

        i = image.copy()

        if not cropping:
            cv2.imshow("image", image)

        elif cropping:
            cv2.rectangle(i, (x_start, y_start), (x_end, y_end), (255, 0, 0), 2)
            cv2.imshow("image", i)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # close all open windows
    cv2.destroyAllWindows()

## Cropping2 - Automatically crops the images to a specified shape of reference image

In [5]:
import cv2
import numpy as np
import os
from PIL import Image

# Specify the directory containing the images
image_dir = 'images/image/'
dest_dir = "cropped_images/"

#specify reference image
ref_img = cv2.imread("images/image/image.png")
m_height, m_width = ref_img.shape[:2]

# Get the filenames of all the images in the directory
image_filenames = os.listdir(image_dir)

# Find the minimum dimensions of all the images
#min_height = np.inf
#min_width = np.inf

for filename in image_filenames:
    img = cv2.imread(os.path.join(image_dir, filename))
    #pil = Image.open(os.path.join(image_dir, filename))
    #display(pil)
    
    height, width = img.shape[:2]
    if height >= m_height and width >= m_width:
        #min_height = 256
        img = img[:m_height, :m_width, :]
        cv2.imwrite(os.path.join(dest_dir, filename[:-4]+".tiff"), img)
        print(img.shape, filename)
    #if width < min_width:
        #min_width = 256
    else:
        continue

# Crop all the images to the same dimensions
#for filename in image_filenames:
    #img = cv2.imread(os.path.join(image_dir, filename))
    #img = img[:min_height, :min_width, :]
    #cv2.imwrite(os.path.join(dest_dir, filename[:-5]+".tiff"), img)


(880, 880, 3) image.png
(880, 880, 3) image15.jpg
(880, 880, 3) image19.jpg
(880, 880, 3) image28.jpg
(880, 880, 3) image9.png


## Slicing 1.1 - Grid type slicing, to generate number of images from a single image

In [1]:
import image_slicer
import os

img_dir = "ex/cropped/"
dest_dir = "ex/sliced/"

#img_dir = "cropped_images/"
#dest_dir = "sliced/"
for filename in os.listdir(img_dir):
    image = os.path.join(img_dir, filename)
    tiles=image_slicer.slice(image, 20, save=False)
    image_slicer.save_tiles(tiles, directory=dest_dir,\
                                prefix=filename[:-5], format='tiff')

## Slicing 1.2 - Slicing using a reference image

In [4]:
import cv2
import os

ref_image = cv2.imread("refimg.tiff")
refh, refw = ref_image.shape[:2]

img_dir = "img/img/"
dest_dir = "img/islice/"

for filename in os.listdir(img_dir):
    path = os.path.join(img_dir, filename)
    new_image= cv2.imread(path)
    
    h, w = new_image.shape[:2]

    start_h = int((h-refh)/2)
    start_w = int((w-refw)/2)

    end_h = start_h + refh
    end_w = start_w + refw
    
    new_image = new_image[start_h:end_h, start_w:end_w, :]
    
    cv2.imwrite(dest_dir+filename[:-4]+".tiff", new_image)

In [9]:
filenames = os.listdir("sliced/")
for filename in filenames:
    img = cv2.imread(os.path.join("sliced/", filename))
    print(img.shape, filename)

(220, 176, 3) image15_01_01.tiff
(220, 176, 3) image15_01_02.tiff
(220, 176, 3) image15_01_03.tiff
(220, 176, 3) image15_01_04.tiff
(220, 176, 3) image15_01_05.tiff
(220, 176, 3) image15_02_01.tiff
(220, 176, 3) image15_02_02.tiff
(220, 176, 3) image15_02_03.tiff
(220, 176, 3) image15_02_04.tiff
(220, 176, 3) image15_02_05.tiff
(220, 176, 3) image15_03_01.tiff
(220, 176, 3) image15_03_02.tiff
(220, 176, 3) image15_03_03.tiff
(220, 176, 3) image15_03_04.tiff
(220, 176, 3) image15_03_05.tiff
(220, 176, 3) image15_04_01.tiff
(220, 176, 3) image15_04_02.tiff
(220, 176, 3) image15_04_03.tiff
(220, 176, 3) image15_04_04.tiff
(220, 176, 3) image15_04_05.tiff
(220, 176, 3) image19_01_01.tiff
(220, 176, 3) image19_01_02.tiff
(220, 176, 3) image19_01_03.tiff
(220, 176, 3) image19_01_04.tiff
(220, 176, 3) image19_01_05.tiff
(220, 176, 3) image19_02_01.tiff
(220, 176, 3) image19_02_02.tiff
(220, 176, 3) image19_02_03.tiff
(220, 176, 3) image19_02_04.tiff
(220, 176, 3) image19_02_05.tiff
(220, 176,

## Creating Masks - Use mouse (scrolling) and keyboards (key d and i) to mask images

### To save press key s and to quit press key q

In [5]:
import os
import cv2
import numpy as np
import glob

# Load the training image

#img_dir = "ex/sliced/"
#mask_dir = "ex/mask/"

img_dir = "img/islice"
mask_dir = "img/imask"

#img_dir = "sliced/"
#mask_dir = "masks/"
os.makedirs(mask_dir, exist_ok=True)

# Define the initial contrast value
contrast_value = 1
i=0

# Define the threshold values for each category
chromosome_threshold = 100
#telomere_threshold = 200
#neither_threshold = 0

# Loop over images in directory
for filename in os.listdir(img_dir):
    # Load image
    img_path = os.path.join(img_dir, filename)
    img = cv2.imread(img_path)
    
    # Define the function to update the image based on the contrast value and threshold values
    def update_img():
        global img, contrast_value, chromosome_threshold, telomere_threshold
        # Convert the image to grayscale
        #gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apply contrast adjustment
        adjusted_img = cv2.convertScaleAbs(img, alpha=contrast_value)
        # Create separate masks for each category based on the threshold values
        telomere_mask = cv2.threshold(adjusted_img, chromosome_threshold, 255, cv2.THRESH_BINARY)[1]
        # Display the image pieces in a grid
        grid_img = np.concatenate((img, telomere_mask), axis=1)
        cv2.imshow('image', grid_img)

    # Create a window to display the image
    cv2.namedWindow('image')

    # Define the function to handle keyboard events
    def keyboard_event(event, x, y, flags, params):
        global contrast_value, chromosome_threshold
        # Adjust the contrast
        if event == cv2.EVENT_MOUSEWHEEL:
            if flags > 0:
                contrast_value += 0.1
            else:
                contrast_value -= 0.1
            update_img()
        # Adjust the threshold values
        else:
            key = cv2.waitKey(0)
            if key == ord('q'):
                cv2.destroyAllWindows()
            elif key == ord('s'):
                adjusted_img = cv2.convertScaleAbs(img, alpha=contrast_value)
                telomere_mask = cv2.threshold(adjusted_img, chromosome_threshold, 255, cv2.THRESH_BINARY)[1]
                
                mask_path = os.path.join(mask_dir, filename[:-5] + "_mask.tiff")
                cv2.imwrite(mask_path, telomere_mask)
                cv2.destroyAllWindows()
                #cv2.imwrite('mask/chromosome_mask/chromosome_mask.png', telomere_mask)
            elif key == ord('d'):
                chromosome_threshold -= 10
                update_img()
            elif key == ord('i'):
                chromosome_threshold += 10
                update_img()

    # Set the mouse callback to handle keyboard events
    #for i in range(len(img)):
    cv2.setMouseCallback('image', keyboard_event)
    #cv2.destroyAllWindows()

    # Display the initial image
    update_img()

    # Wait for user input
    cv2.waitKey(0)


# Cleanup
cv2.destroyAllWindows()
